In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab07.ipynb")

# Lab 7: Simple Linear Regression

### Objective

In this lab, we will review some of the details of how linear regression works as described in Lectures 12 and 13.

We will also show you how to do linear regression using various real world tools including:
- (Question 1) `seaborn`
- (Question 2) analytic solution
- (Question 3) `scipy.optimize`
- (Question 4 & 5) `scikit-learn`

In real world data science work, you are far more likely to use something similar to the `seaborn` and `scikit-learn` approaches, but it is important to know how to use the formulaic and `scipy.optimize` approaches so that you understand what is really going on.

**This assignment should be completed and submitted by 11:59 PM PDT on Tuesday, October 12th.**


### Collaboration Policy

Data science is a collaborative activity. While you may talk to others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** in the following cell:

_List collaborators here_

In [1]:
# Run this cell
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Run this cell to suppress all DeprecationWarnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

We begin by importing the tips dataset that we also explored in Lab 6.

In [3]:
tips = sns.load_dataset("tips")

In [4]:
tips.head(5)

In Lab 6, we fit a **constant** model to this dataset, meaning our model was $\hat{y} = \theta$. In other words, given the set of tips `tips['tip']`, we tried to find a summary statistic $\theta$ that best represented our set of tips. To find the value of $\theta$, we minimized the following empirical risk:

$$R(\theta) = \frac{1}{n}\sum_{i = 1}^n L(y_i, \theta)$$

Here, $\mathcal{D} = \{y_1, y_2, ..., y_n \}$ refers to our set of `tips` values.

We looked at two different loss functions:

- $L_2(y_i, \hat{y_i}) = (y_i - \hat{y_i})^2$

- $L_1(y_i, \hat{y_i}) = \left| y_i - \hat{y_i} \right|$


<br>

By contrast, in this lab, we're interested in studying the **relationship between two variables**. Specifically, we're interested in the relationship between the `total_bill` column and `tip` column. Our goal will be to predict tip ($y$) from total_bill ($x$), i.e., we want to find values of $a$ and $b$ so that given $x$, predict $y$ as
$$\boxed{\hat{y} = a + bx}$$
We will now explore different ways to obtain the optimal values of $a, b$, called $\hat{a}, \hat{b}$, where $\hat{y} = \hat{a} + \hat{b}x$.

First, let's run `sns.lmplot`, which will both provide a scatterplot of `tip` vs `total_bill` and display the least-squares line of best fit. We will look into solving for the line of best fit in three different ways: manually using the formula from lecture, `scipy.optimize`, and `scikit-learn`.

In [5]:
sns.lmplot(data = tips, x = "total_bill", y = "tip");

## Question 1 – Analytic Solution

In [Lecture 12](http://ds100.org/sp21/lecture/lec12/), we derived the following expression for the line of best fit.

$$\hat{y_i} = \bar{y} + r \frac{SD(y)}{SD(x)} (x_i - \bar{x})$$

where $\bar{x}$, $\bar{y}$, $SD(x)$, $SD(y)$ correspond to the means and standard deviations of $x$ and $y$, respectively, and $r$ is the correlation coefficient.

### Question 1a

Assign `x_bar`, `y_bar`, `std_x`, `std_y`, and `r`, for our dataset. Note: Make sure to use `np.std`, and not `<Series name>.std()`.

- Hint: Remember, in our case, `y` is `tip`, and `x` is `total_bill`.
- Hint: You may find `np.corrcoef` handy in computing `r`. Note that the output of `np.corrcoef` is a matrix, not a number, so you'll need to collect the correlation coefficient by indexing into the returned array.

<!--
BEGIN QUESTION
name: q1a
-->

In [6]:
x_bar = ...
y_bar = ...
std_x = ...
std_y = ...
r = ...

In [ ]:
grader.check("q1a")

### Question 1b

Now, set `b_hat` and `a_hat` correctly, in terms of the variables you defined above. 

- Hint: Try and match the slope and intercept in $\hat{y_i} = \hat{a} + \hat{b}x_i$ to the slope and intercept in $\hat{y_i} = \bar{y} + r \frac{SD(y)}{SD(x)} (x_i - \bar{x})$.

- Hint: You may want to define `a_hat` in terms of `b_hat`.

<!--
BEGIN QUESTION
name: q1b
-->

In [13]:
b_hat = ...
a_hat = ...

In [ ]:
grader.check("q1b")

### Question 1c

Now, use `a_hat` and `b_hat` to predict the tip for a total bill amount of $20. Store your result in `predicted_20`.

<!--
BEGIN QUESTION
name: q1c
-->

In [16]:
predicted_20 = ...
predicted_20

In [ ]:
grader.check("q1c")

### Question 1d
Assign `regression` to be a `pd.Series` of predicted $y$ values (i.e., predicted `"tip"` values) for the observed total bills (`tips["total_bill"]`). You will need to use `a_hat`, `b_hat`, and `tips["total_bill"]`.

<!--
BEGIN QUESTION
name: q1d
-->

In [18]:
regression = ...

In [ ]:
grader.check("q1d")

If you defined `regression` correctly, the following cell will generate a scatter plot of `tip` vs. `total_bill`, along with the line of best fit you just computed.

In [20]:
sns.scatterplot(x='total_bill', y='tip', data=tips)
plt.plot(tips["total_bill"], regression, color = 'r')
plt.xlabel('total_bill')
plt.ylabel('tip');

### Question 1e

Consider $r$, the correlation coefficient between `tips` and `total_bill`.

In [21]:
r

**In the cell below**, comment on the value of $r$, and what it means in the context of the above scatter plot.
<!--
BEGIN QUESTION
name: q4b
-->

**YOUR ANSWER HERE**

_Type your answer here, replacing this text._

## Question 2 – Using Scipy Minimize

`scipy.minimize` is a powerful method that can determine the optimal value of a variety of different functions. In practice, it is used to minimize functions that have no (or difficult to obtain) analytical solutions (it is a **numerical method**).

It is overkill for our simple example, but nonetheless, we will show you how to use it, as it will become useful in the near future.

First, we define the function `l2_tip_risk` which computes the empirical risk for a given choice of `a` and `b`. That is, it computes

$$R(a, b) = \frac{1}{n} \sum_{i = 1}^n(y_i - (a + b x_i))^2$$

where, again, $x$ and $y$ refer to `"total_bill"` and `"tip"`.

As a reminder: Risk refers to the average loss on the training set.


In [22]:
def l2_tip_risk(a, b):
    """Returns average l2 loss between regression line for intercept a 
       and slope b"""
    y_hat = a + b * tips["total_bill"]
    return np.mean((tips["tip"] - y_hat)**2)

Try out different `a` and `b` values. Observe that if you pick values close to the ones from the earlier part of this lab then the risk is lower. 

In [23]:
l2_tip_risk(0.9, 0.1)

The `minimize` function we saw in Lab 6 can also minimize functions of multiple variables. There's one quirk, however, which is that the function has to accept its parameters as a single list.

For example, consider the multivariate $f(u, v) = u^2 - 2 u v - 3 v + 2 v^2$. It turns out this function's minimum is at $(1.5, 1.5)$. To minimize this function, we create `f`.

In [24]:
def f(theta):
    u = theta[0]
    v = theta[1]
    return u**2 - 2 * u * v - 3 * v + 2 * v**2

In [25]:
from scipy.optimize import minimize
minimize(f, x0 = [0.0, 0.0]) 

# As an aside: x0 is the "initial guess" for the optimal theta. Minimize iteratively updates theta.
# We will study an iterative algorithm for function minimization in the coming weeks.

### Question 2a

Define `l2_tip_risk_list` which is exactly like `l2_tip_risk` except that it takes in a single list of 2 variables rather than two separate variables. For example `l2_tip_risk_list([2, 3])` should return the same value as `l2_tip_risk(2, 3)`.

<!--
BEGIN QUESTION
name: q2
-->

In [26]:
def l2_tip_risk_list(theta):
    """Returns average l2 loss between regression line for intercept a 
       and slope b"""
    ...

In [ ]:
grader.check("q2")

### Question 2b

Now, set `minimized` to the result of calling `minimize` to optimize this risk function.

- Hint: Make sure to set `x0`.

<!--
BEGIN QUESTION
name: q2b
-->

In [28]:
minimized = ...

Let's look at the output of your call to `minimize`.

In [29]:
minimized

The following cell will print out the values of `a_hat` and `b_hat` computed from both methods ("manual" refers to the technique in Question 1). If you've done everything correctly, these should be very close to one another.

In [30]:
print('a_hat_scipy: ', minimized['x'][0])
print('a_hat_manual: ', a_hat)
print('\n')
print('b_hat_scipy: ', minimized['x'][1])
print('b_hat_manual: ', b_hat)

The reason these don't match past the first 5 decimal places is due to the fact that `scipy.minimize` is a numerical method, meaning it approximates the optimal value using an iterative procedure. You shouldn't worry about the tiny differences in solutions from different runs of `minimize`

## Question 3 – Using Scikit Learn

Yet another way to fit a linear regression model is to use scikit learn, an industry standard package for machine learning applications. 

To do so, we first create a `LinearRegression` object.

In [31]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

Here, `model` is like a "blank slate" for a linear model. Now, we need to tell `model` to "fit" itself to the data. Essentially, this is doing exactly what you did in the previous part of this lab (creating a risk function and finding the parameters that minimize that risk).

<i>Note: `X` needs to be a matrix (or DataFrame), as opposed to a single array (or Series). This is because `sklearn.linear_model` is robust enough to be used for multiple regression, which we will look at in Question 4.</i>

In [32]:
model.fit(X = tips[['total_bill']], y= tips['tip'])

Now that the model exists, we can look at the a_hat and b_hat values it found, which are given in the attributes `intercept` and `coef`, respectively.

In [33]:
model.coef_

In [34]:
model.intercept_

To use the `scikit-learn` linear regression model to make predictions, you can use the `model.predict` method:

In [35]:
model.predict([[20]]) # 20 needs to be a 2D array since the X above was a 2D array.

The above line of code tells us that `model` predicts a tip of $\$3.02$ given a total bill amount of $\$20$. This is the same as doing `a_hat + b_hat * 20` as in Question 1c.

### Question 3a

Create a linear regression plot using `model.predict`. It should look very similar (if not the same) as your plot from Question 1d.

<!--
BEGIN QUESTION
name: q3a
-->

In [36]:
predicted_tip = ...
sns.scatterplot(x='total_bill', y='tip', data=tips)
plt.plot(tips["total_bill"],  predicted_tip, color = 'r');

## Question 4 – Multiple Linear Regression

In the previous parts we showed how to establish relationships between one independent explanatory variable and one response variable. However, with real-world problems you will often want to use **multiple features** to model and predict a response variable. To do so, we will use multiple linear regression, as discussed in [Lecture 13](http://ds100.org/sp21/lecture/lec13/). Multiple linear regression attempts to model the relationship between two or more explanatory variables and a response variable by fitting a linear equation to the observed data. Formally, the model for multiple linear regression, given $p$ features is:

$$y_i = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + … + \theta_p x_p $$

Please note that we have been using the terms **features**, **independent variables**, and **explanatory variables** interchangeably. Usually “features” are used in the context of machine learning when you are trying to make predictions. “Independent variables” and “explanatory variables” are mainly found in statistics, econometrics and other related fields which focus on understanding the relationship between a set of variables.  


For example, consider the plot below which shows fuel efficiency vs. engine power for several models of automobiles.

In [37]:
# Here, we load the fuel dataset, and drop any rows that have missing data
vehicle_data = sns.load_dataset('mpg').dropna()
vehicle_data = vehicle_data.sort_values('horsepower', ascending=True)
vehicle_data.head(5)

In [38]:
sns.scatterplot(x='horsepower', y='mpg', data=vehicle_data);

If we use `horsepower` alone to predict `mpg`, we get not-so-great results.

In [39]:
sns.lmplot(x='horsepower', y='mpg', data=vehicle_data);

Below, we make a residual plot of the residuals versus the fitted values for this simple linear regression model.

In [40]:
model_simple = LinearRegression()
model_simple.fit(vehicle_data[['horsepower']], vehicle_data['mpg'])
model_simple_fitted = model_simple.predict(vehicle_data[['horsepower']])
plt.scatter(model_simple_fitted, vehicle_data['mpg'] - model_simple_fitted)
plt.xlabel('Fitted Values')
plt.ylabel('Residuals');

This is not an example of a "good" residual plot. There is an underlying parabolic pattern in the residuals, so we should consider adding a quadratic feature.

We can consider including functions of existing features as new features to help improve the predictive power of our model. (This is something we will discuss in further detail in the Feature Engineering lecture, Lecture 14.) For example, the line below adds a column which contains the square of the horsepower for each car in the dataset.

In [41]:
vehicle_data['hp^2'] = vehicle_data['horsepower'] ** 2
vehicle_data.head()

### Question 4a

Using scikit learn's `LinearRegression`, create and fit a model that tries to predict `mpg` from `horsepower` AND `hp^2`. Name your model `model_multiple`.

- Hint: We do something very similar in Question 3.

<!--
BEGIN QUESTION
name: q4a
-->

In [42]:
model_multiple = LinearRegression()
...

In [ ]:
grader.check("q4a")

After fitting, we can see the coefficients and intercept. Note, there are now two elements in `model_multiple.coef_`, since there are two features.

In [45]:
model_multiple.coef_

In [46]:
model_multiple.intercept_

### Question 4b

Using the above values, in LaTeX, write out the function that the model is using to predict `mpg` from `horsepower` and `hp^2`.

<!--
BEGIN QUESTION
name: q4b
-->

_Type your answer here, replacing this text._

The plot below shows the prediction of our model. It's much better!

In [47]:
# DO NOT CHANGE THIS CELL
predicted_mpg = model_multiple.predict(vehicle_data[['horsepower', 'hp^2']])
sns.scatterplot(x='horsepower', y='mpg', data=vehicle_data)
plt.plot(vehicle_data['horsepower'],  predicted_mpg, color = 'r');

### Question 4c

In the cell below, explain why we use the term "linear" to describe the model above, even though it incorporates horsepower squared as a feature.

<!--
BEGIN QUESTION
name: q4c
-->

_Type your answer here, replacing this text._

To see exactly how much better our new model is, we can compare the Multiple $R^2$ from these two fits.  As described in Lecture 12,


$$R^2 = \frac{\text{variance of fitted values}}{\text{variance of true } y} = \frac{\sigma_{\hat{y}}^2}{\sigma_y^2}$$

Unlike $r$, the correlation coefficient we looked at in Question 1, $R^2$  can be used
in the multiple regression setting.  In simple regression, $r^{2}$ and Multiple $R^{2}$ are
the same.

In [48]:
predicted_mpg_horsepower_only = (
    LinearRegression()
    .fit(X = vehicle_data[['horsepower']], y = vehicle_data['mpg'])
    .predict(vehicle_data[['horsepower']])
)

r2_horsepower_only = np.var(predicted_mpg_horsepower_only) / np.var(vehicle_data['mpg'])
r2_both = np.var(predicted_mpg) / np.var(vehicle_data['mpg'])

print('Multiple R^2 using only horsepower: ', r2_horsepower_only)
print('Multiple R^2 using both horsepower and horsepower squared: ', r2_both)

By introducing `hp^2` as a feature, our multiple $R^2$ value increased. Think about what this means with respect to the strength of our refined model.


### Question 4d

In the cell below, we assign the mean of the `mpg` column of the `vehicle_data` dataframe to `mean_mpg`. Given this information, what is the mean of the `predicted_mpg` and `predicted_mpg_horsepower_only` arrays?

Hint: You should not have to call `np.mean` in your code.

<!--
BEGIN QUESTION
name: q4d
-->

In [49]:
mean_mpg = np.mean(vehicle_data['mpg'])
mean_predicted_mpg = ...
mean_predicted_mpg_horsepower_only = ...

In [ ]:
grader.check("q4d")

## Question 5

Let's take this one step further, and introduce a few more features.


Again, using scikit learn's `LinearRegression`, we fit a model that tries to predict `mpg` using each of the following as features:
- `horsepower`
- `hp^2`
- `model_year`
- `acceleration`



In [52]:
desired_columns = ['horsepower', 'hp^2', 'model_year', 'acceleration']
model_many = LinearRegression()
model_many.fit(X = vehicle_data[desired_columns], y= vehicle_data['mpg'])

The plot below shows the prediction of our more sophisticated model.

In [53]:
predicted_mpg_many = model_many.predict(vehicle_data[['horsepower', 'hp^2', 'model_year', 'acceleration']])
sns.scatterplot(x='horsepower', y='mpg', data=vehicle_data)
plt.plot(vehicle_data['horsepower'],  predicted_mpg_many, color = 'r');

Think about what you see in the above plot. Why is the shape of our prediction curve so jagged? Do you think this is a good model to predict the `mpg` of some car we don't already have information on?

This idea – the **bias-variance tradeoff** – is an idea we will explore in the coming weeks.

### Question 5a

Lastly, set `r2_many` to be the multiple $R^2$ coefficient obtained by using `model_many`.

- Hint: This is very similar to what we did right before Question 4d. Use `predicted_mpg_many`.
- Note: The above plotting cell needs to have been run in order for this to work.

<!--
BEGIN QUESTION
name: q5a
-->

In [54]:
r2_many = ...

In [ ]:
grader.check("q5a")

In [56]:
print('Multiple R^2 using only horsepower: ', r2_horsepower_only)
print('Multiple R^2 using both horsepower and horsepower squared: ', r2_both)
print('Multiple R^2 using horsepower, horsepower squared, model year, and acceleration: ', r2_many)

If everything was done correctly, the multiple $R^2$ of our latest model should be substantially higher than that of the previous two models. This is because multiple $R^2$ increases with the number of covariates we add to our model. 

We might not always want to use models with large multiple $R^2$ values because these models could be overfitting to the training data, and won't generalize well to unseen data. Again, this is an idea we will explore in future lectures and assignments.

# Congrats, you finished the lab!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)